In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("/kaggle/input/ckdisease/kidney_disease.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.drop(["id"], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.columns = ['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar', 'red_blood_cells', 'pus_cell', 'puss_cell_clumbs', 'bacteria', 'blood_glucose_random', 'blood_urea',
       'serum_creatinine', 'sodium', 'potassium', 'hemoglabin', 'packed_cell_volume', 'wide_blood_cell_count', 'red_blood_cell_count', 'hypertension', 'diabetes_mellitus', 'coronary_artery_disease',
       'appetite', 'peda_edema', 'anemia', 'class']

In [ ]:
df.columns

In [ ]:
len(df.columns)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df["packed_cell_volume"]

In [ ]:
df["packed_cell_volume"] = pd.to_numeric(df["packed_cell_volume"], errors = "coerce")
df["wide_blood_cell_count"] = pd.to_numeric(df["wide_blood_cell_count"], errors = "coerce")
df["red_blood_cell_count"] = pd.to_numeric(df["red_blood_cell_count"], errors = "coerce")

In [ ]:
df.info()

In [ ]:
categorical_cols = [col for col in df.columns if df[col].dtype == "object"]

In [ ]:
categorical_cols

In [ ]:
len(categorical_cols)

In [ ]:
num_cols = [col for col in df.columns if df[col].dtype != "object"]

In [ ]:
len(num_cols)

In [ ]:
for col in categorical_cols:
    print(f"{col}: {df[col].unique()}")

In [ ]:
df["diabetes_mellitus"].replace(to_replace = {" yes":"yes", "\tno":"no", "\tyes":"yes"}, inplace=True)
df["coronary_artery_disease"].replace(to_replace = {"\tno":"no"}, inplace=True)
df["class"].replace(to_replace = {"ckd\t":"ckd"}, inplace=True)

In [ ]:
for col in categorical_cols:
    print(f"{col}: {df[col].unique()}")

In [ ]:
df["class"] = df["class"].map({"ckd":0, "notckd":1})

In [ ]:
df["class"]

In [ ]:
plt.figure(figsize=(20,20))
plotnumber = 1
for col in num_cols:
    if plotnumber <= 14:
        ax = plt.subplot(3, 5, plotnumber)
        sns.histplot(df[col], kde=True)
        plt.xlabel(col)
    plotnumber += 1
plt.tight_layout()
plt.show()

In [ ]:
float_df = df.select_dtypes(include=['float64','int64'])
plt.figure(figsize=(15,15))
sns.heatmap(float_df.corr(), annot=True, linecolor="white", linewidths=2, cmap="magma")
plt.show()

In [ ]:
df.info()

In [ ]:
def kde(col):
    grid = sns.FacetGrid(df, hue="class", height=6, aspect=2)

    grid.map(sns.kdeplot, col)
    grid.add_legend()
kde("hemoglabin")

In [ ]:
kde("wide_blood_cell_count")

In [ ]:
kde("packed_cell_volume")

In [ ]:
kde("red_blood_cell_count")

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
def solve_mv_random_value(feature):
    random_sample = df[feature].dropna().sample(df[feature].isnull().sum())
    random_sample.index = df[df[feature].isnull()].index
    df.loc[df[feature].isnull(), feature] = random_sample

In [ ]:
for col in num_cols:
    solve_mv_random_value(col)

In [ ]:
df[num_cols].isnull().sum()

In [ ]:
def solve_mv_mode(feature):
    mode = df[feature].mode()[0]
    df[feature] = df[feature].fillna(mode)

In [ ]:
solve_mv_random_value("red_blood_cells")
solve_mv_random_value("pus_cell")

In [ ]:
for col in categorical_cols:
    solve_mv_mode(col)

In [ ]:
df[categorical_cols].isnull().sum()

In [ ]:
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()}")

In [ ]:
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

In [ ]:
df

In [ ]:
df.info()

In [ ]:
independent_col = [col for col in df.columns if col != "class"]

In [ ]:
independent_col

In [ ]:
dependent_col = "class"

In [ ]:
X = df[independent_col]
y = df[dependent_col]

In [ ]:
y

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
dtc_acc = accuracy_score(y_test, y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

In [ ]:
cr = classification_report(y_test, y_pred)

In [ ]:
print("Classification Report", cr)

In [ ]:
class_names = ["ckd", "notckd"]
plt.figure(figsize=(20,10))
plot_tree(dtc, feature_names = independent_col, filled=True, rounded=True, fontsize=10)
plt.show()

In [ ]:
feature_importance = pd.DataFrame({"Feature":independent_col, "Importance":dtc.feature_importances_})

In [ ]:
feature_importance.sort_values(by="Importance", ascending=False)

In [ ]:
print("The Most Important Feature: ", feature_importance.sort_values(by="Importance", ascending=False).iloc[0])

In [ ]:
plt.figure()
sns.barplot(x = "Importance", y = "Feature", data = feature_importance)
plt.title("Feature Importance")
plt.show()